In [1]:
import numpy as np
from numpy import linalg as LA
import os
import copy
import ase
import ase.io
from ase.units import Ha, Bohr, mol,kcal
from ase.io import write
from ase.io.trajectory import Trajectory

In [2]:
basis='cc-pvtz'
xc='lda,lda'; method = 'rks'
# xc=None; method = 'mp2'

# mdtraj = 'h2o/water_400_train.traj'
mdtraj = 'h2o/water_300_train.traj'
mdtraj2 = 'h2o/water_300.traj'

nsamples = 30
tol = 0.015
direction=(0,0,1)

skip = 0
nseq = 100
nstep = 100

input_options={
    'basis': basis,
    'xc': xc,
    'method': method,
    'tol': tol,
    'direction': direction,
    'skip': skip,
    'nseq': nseq,
    'nstep': nstep,
    'train_mdtraj': mdtraj,
    'test_mdtraj': mdtraj2,
}

In [3]:
from qmlearn.model import QMModel
from qmlearn.drivers.mol import QMMol
from qmlearn.preprocessing import get_train_atoms
from qmlearn.io.hdf5 import DBHDF5

In [4]:
train_atoms=get_train_atoms(mdtraj=mdtraj, nsamples=nsamples, tol=tol, direction=direction)
nsamples = len(train_atoms)

WARN : Only get 25 samples at 7170 step. Maybe you can reduce the 'tol'.


In [5]:
import os
outtraj='_QML_selected'.join(os.path.splitext(mdtraj))
train_traj = Trajectory(outtraj, 'w')
for atoms in train_atoms:
    train_traj.write(atoms)
train_traj.close()

In [6]:
refqmmol = QMMol(atoms = train_atoms[0],  method = method, basis=basis, xc = xc)

In [7]:
prop = ['vext', 'gamma', 'energy', 'ke', 'dipole', 'quadrupole', 'ncharge', 'idm']
properties= { k: [] for k in prop}
test_data = { k: [] for k in prop}

In [8]:
def gamma2props(qmmol, gamma, p, prop=[]):
    if 'vext' in prop:
        p['vext'].append(qmmol.engine.vext)
    if 'gamma' in prop:
        p['gamma'].append(gamma)
    if 'energy' in prop:
        p['energy'].append(qmmol.calc_etotal(gamma))
    if 'ke' in prop:
        p['ke'].append(qmmol.calc_ke(gamma))
    if 'dipole' in prop:
        p['dipole'].append(qmmol.calc_dipole(gamma))
    if 'quadrupole' in prop:
        p['quadrupole'].append(qmmol.calc_quadrupole(gamma))
    if 'ncharge' in prop:
        p['ncharge'].append(qmmol.calc_ncharge(gamma))
    if 'idm' in prop:
        p['idm'].append(qmmol.calc_idempotency(gamma))
    return p

In [9]:
%%capture
for atoms in train_atoms:
    qmmol = refqmmol.duplicate(atoms)
    qmmol.run()
    gamma=qmmol.engine.gamma
    gamma2props(qmmol, gamma, properties, prop=prop[:-1])
    properties['energy'][-1] = qmmol.engine.etotal

In [10]:
%%capture
test_atoms = []
traj = Trajectory(mdtraj2)
for i in range(0,nstep):
    j = skip+i*nseq
    atoms=traj[j]
    test_atoms.append(atoms)
    qmmol = refqmmol.duplicate(atoms)
    qmmol.run()
    p = test_data
    gamma = qmmol.engine.gamma
    gamma2props(qmmol, gamma, p, prop=prop[:-1])
    p['energy'][-1] = qmmol.engine.etotal
traj.close()

In [11]:
dbfile = os.path.splitext(mdtraj2)[0]+'_QML_set.hdf5'
db = DBHDF5(dbfile, qmmol=refqmmol)

In [12]:
properties['input_options']=input_options
db.write_qmmol(refqmmol)
db.write_images(train_atoms, prefix='train')
db.write_properties(properties, prefix='train')
db.write_images(test_atoms, prefix='test')
db.write_properties(test_data, prefix='test')
db.names

['mp2',
 'mp2/qmmol',
 'mp2/test_atoms_100',
 'mp2/test_props_100',
 'mp2/train_atoms_25',
 'mp2/train_props_25',
 'rks',
 'rks/qmmol',
 'rks/test_atoms_100',
 'rks/test_props_100',
 'rks/train_atoms_25',
 'rks/train_props_25']

In [13]:
db.close()

In [15]:
db = DBHDF5(dbfile, qmmol=refqmmol)
data = db.read_properties(db.get_names('*/test_props_*')[0])
db.close()
data['energy'][:3]

array([-76.25270517, -76.26103005, -76.25225214])